<a href="https://colab.research.google.com/github/DanZter/BERT_framework/blob/master/BERT_on_steroids_X_singlecore_tpu_training_spawn_(time).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 753.2MB 19kB/s 
     |████████████████████████████████| 6.6MB 46.1MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101


In [3]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  22741      0 --:--:-- --:--:-- --:--:-- 22741
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
Uninstalling torch-1.5.1:
  Successfully uninstalled torch-1.5.1
     |████████████████████████████████| 61kB 2.3MB/s 
Uninstalling torchvision-0.6.1:
  Successfully uninstalled torchvision-0.6.1
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Trace

In [4]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.26.177.146:8470']
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
!pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 3.0MB 19.8MB/s 
     |████████████████████████████████| 890kB 33.8MB/s 
     |████████████████████████████████| 1.1MB 43.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e6a1471690a7ce9f52238b00a0864d83afcf4f11395e3aaba498ed548768de19
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
# pip uninstall torchvision-nightly-cp36-cp36m-linux_x86_64.whl -y

In [8]:
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn import model_selection
from scipy import stats
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [10]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 30)       # pool output has 768 features, 30 targets

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)  # not using o1 - Sequential Output
        bo = self.bert_drop(o2)                                                     # using o2 - Pooled Output
        return self.out(bo)

class BERTDatasetTraining:
    def __init__(self, qtitle, qbody, answer, targets, tokenizer, max_len):
        self.qtitle = qtitle
        self.qbody = qbody
        self.answer = answer
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targets = targets                   # numpy array of size: no. of samples * 30(targets)

    def __len__(self):
        return len(self.answer)                  # returns length of title or body or answer

    def __getitem__(self, item):                 # takes in index and returns output
        question_title = str(self.qtitle[item])
        question_body = str(self.qbody[item])
        answer = str(self.answer[item])

        # [CLS] [Q-TITLE] [Q-BODY] [SEP] [ANSWER] [SEP]

        inputs = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            answer,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True 
        )

        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        padding_len = self.max_len - len(ids)
        ids = ids + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.targets[item, :], dtype=torch.long)
        }

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        # optimizer.step()                                         # cuda
        xm.optimizer_step(optimizer)                              # tpu
        if scheduler is not None:
            scheduler.step()
        if bi == 0:
            print(f"batch index={bi}, loss={loss}")
    print(f"batch index={bi}, loss={loss}")

def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)

        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())        # Linear Layer: can apply sigmoid here

        return np.vstack(fin_outputs), np.vstack(fin_targets)

def run(index):
    MAX_LEN = 512
    TRAIN_BATCH_SIZE = 4
    EPOCHS = 20

    dfx = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_train.csv").fillna("none")
    df_train, df_valid = model_selection.train_test_split(dfx, random_state = 42, test_size = 0.1)
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    sample = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_sample_submission.csv")
    target_cols = list(sample.drop("qa_id", axis=1).columns)
    train_targets = df_train[target_cols].values
    valid_targets = df_valid[target_cols].values

    tokenizer =transformers.BertTokenizer.from_pretrained("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased")

    train_dataset = BERTDatasetTraining(
        qtitle=df_train.question_title.values,
        qbody=df_train.question_body.values,
        answer=df_train.answer.values,
        targets=train_targets,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
    train_sampler = torch.utils.data.DistributedSampler(
       train_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal(),
       shuffle=True 
    )
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size= TRAIN_BATCH_SIZE,
        sampler=train_sampler
    )
    ############
    valid_dataset = BERTDatasetTraining(
        qtitle=df_valid.question_title.values,
        qbody=df_valid.question_body.values,
        answer=df_valid.answer.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
    valid_sampler = torch.utils.data.DistributedSampler(
       valid_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal()
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=4,
        sampler=valid_sampler
    )

    # device = "cuda"                   # cuda
    device = xm.xla_device()            # tpu
    lr = 3e-5 *xm.xrt_world_size()
    num_train_steps = int(len(train_dataset)/ TRAIN_BATCH_SIZE / xm.xrt_world_size() * EPOCHS)
    model = BERTBaseUncased("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased").to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    for epoch in tqdm(range(EPOCHS), total=EPOCHS):
      print("X"*20, "EPOCH :", epoch, "X"*100)
      para_loader = pl.ParallelLoader(train_data_loader, [device])
      train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler)

      para_loader = pl.ParallelLoader(valid_data_loader, [device])
      o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)

      spear = []
      for jj in range(t.shape[1]):
          p1 = list(t[:,jj])
          p2 = list(o[:, jj])
          coef, _ = np.nan_to_num(stats.spearmanr(p1,p2))
          spear.append(coef)
      spear = np.mean(spear)
      print(f"epoch = {epoch}, spearman = {spear}")
      # torch.save(model.state_dict(), "model.bin")         # cuda
      # xm.save(model.state_dict(), "model.bin")          # tpu

if __name__=="__main__":
  xmp.spawn(run, nprocs=1)



  0%|          | 0/20 [00:00<?, ?it/s]

XXXXXXXXXXXXXXXXXXXX EPOCH : 0 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
batch index=0, loss=0.7307649850845337
batch index=1367, loss=0.26072824001312256



  5%|▌         | 1/20 [04:05<1:17:37, 245.13s/it]

epoch = 0, spearman = 0.2191600921812487
XXXXXXXXXXXXXXXXXXXX EPOCH : 1 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
batch index=0, loss=0.3421700894832611
batch index=1367, loss=0.2272823005914688



 10%|█         | 2/20 [07:55<1:12:12, 240.70s/it]

...


 95%|█████████▌| 19/20 [1:12:58<03:50, 230.69s/it]

epoch = 18, spearman = 0.20194683286477128
XXXXXXXXXXXXXXXXXXXX EPOCH : 19 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
batch index=0, loss=0.20339158177375793
batch index=1367, loss=0.003992369864135981



100%|██████████| 20/20 [1:17:11<00:00, 231.59s/it]

epoch = 19, spearman = 0.2191600921812487


SystemExit: ignored